In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os, random, torch, time, copy
import torch.nn.functional as F
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset
from PIL import Image, ImageOps, ImageFilter
from skimage.filters import threshold_local

from torch.autograd import Variable

In [ ]:
path='../input/covidunsa'

In [ ]:
classes = ['no-covid', 'covid']


In [ ]:
class image_dataset(Dataset):
    """Class creator for dataset."""

    def __init__(self, csv_path, root_dir, transform=None, phase=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform
        # If not a PA view, drop the line 
        self.df.drop(self.df[self.df.view != 'PA'].index, inplace=True)
        self.phase = phase

    def __len__(self):
        
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        if self.df['finding'].iloc[idx] != 'COVID-19':
            finding = 0
            img_path = os.path.sep.join([path, 'images', self.df['filename'].iloc[idx]])
            image = Image.open(img_path)
            sample = {'image': image, 'finding': finding}
            
            if self.transform:
                sample = {'image': self.transform[self.phase](sample['image']), 'finding': finding}

        else:
            finding = 1
            img_path = os.path.sep.join([path, 'images', self.df['filename'].iloc[idx]])
            image = Image.open(img_path)
            sample = {'image': image, 'finding': finding}

            if self.transform:
                sample = {'image': self.transform[self.phase](sample['image']), 'finding': finding}

        return sample

In [ ]:
xray_dataset = image_dataset(csv_path=os.path.sep.join([path, 'metadata.csv']), root_dir=path)

In [ ]:
Nr = 4
Nc = 3

fig, axs = plt.subplots(Nr, Nc,figsize=(20,10))
for i in range(Nr*Nc):
    sample = xray_dataset[
        i + random.randrange(
            0, len(xray_dataset)-(Nr*Nc)
        )]
    
    plt.subplot(Nr, Nc, i+1, aspect='auto')
    plt.tight_layout()
    plt.title('Finding: '+classes[sample['finding']])
    plt.axis('off')
    plt.imshow(sample['image'], cmap='gray')

plt.show()

In [ ]:
metada=pd.read_csv(path+'/metadata.csv')

In [ ]:
metada[0:-1]

In [ ]:
# dataset_path = './dataset'
covid_dataset_path = '../input/covidunsa'

In [ ]:
metadata=pd.read_csv(covid_dataset_path+"/metadata.csv")